In [1]:
# Start GEE and generate authentication token

In [2]:
# Import the Earth Engine Python Package
import ee

# Earth engine needs to be authenticated before use
ee.Authenticate()

# Initialize the Earth Engine object, using the authentication credentials.
ee.Initialize()

#used to display the images
EE_TILES = 'https://earthengine.googleapis.com/map/{mapid}/{{z}}/{{x}}/{{y}}?token={token}'

Enter verification code: 4/1AY0e-g7Jo8e95CiLfGjyu-TPogox69NxAGZOBoQwiz-1y7tswplCJVdmmYc

Successfully saved authorization token.


In [3]:
# *Folium* is the module used to display the sentinel 2 images

In [4]:
import folium

In [5]:
# A collection of images of the farm from 2018. Watch out, clouds are the enemy 

In [6]:
image00_2018 = ee.Image('COPERNICUS/S2/20180814T081601_20180814T084139_T34HCJ')
image01_2018 = ee.Image('COPERNICUS/S2/20180819T081559_20180819T084531_T34HCJ') # sort of cloudy
image02_2018 = ee.Image('COPERNICUS/S2/20180824T081601_20180824T084030_T34HCJ')
image03_2018 = ee.Image('COPERNICUS/S2/20180829T081959_20180829T084506_T34HCJ')
image04_2018 = ee.Image('COPERNICUS/S2/20180903T081601_20180903T084616_T34HCJ')
image05_2018 = ee.Image('COPERNICUS/S2/20180908T081559_20180908T084807_T34HCJ')
image06_2018 = ee.Image('COPERNICUS/S2/20180913T081601_20180913T083937_T34HCJ') # cloudy
image07_2018 = ee.Image('COPERNICUS/S2/20180918T081559_20180918T084550_T34HCJ')
image08_2018 = ee.Image('COPERNICUS/S2/20180923T081641_20180923T084105_T34HCJ')
image09_2018 = ee.Image('COPERNICUS/S2/20180928T081709_20180928T083759_T34HCJ') # cloudy
image10_2018 = ee.Image('COPERNICUS/S2/20181003T081751_20181003T083901_T34HCJ')
image11_2018 = ee.Image('COPERNICUS/S2/20181008T081819_20181008T083928_T34HCJ')
image12_2018 = ee.Image('COPERNICUS/S2/20181013T081901_20181013T084809_T34HCJ')
image13_2018 = ee.Image('COPERNICUS/S2/20181018T081929_20181018T083838_T34HCJ')
image14_2018 = ee.Image('COPERNICUS/S2/20181023T082001_20181023T084045_T34HCJ')
image_colletion_2018 = [image00_2018, image01_2018,image02_2018, image03_2018, image04_2018, image05_2018, 
                        image08_2018, image10_2018, image11_2018, image12_2018, image13_2018, image14_2018]

In [7]:
# This method is used to display the sentinel images

In [8]:
def Mapdisplay(center, dicc, Tiles="OpensTreetMap",zoom_start=10, markers=False,  points=None):
    '''
    :param center -> Center of the map (Latitude and Longitude).
    :param dicc:  -> Earth Engine Geometries or Tiles dictionary
    :param Tiles: -> Mapbox Bright,Mapbox Control Room,Stamen Terrain,Stamen Toner,stamenwatercolor,cartodbpositron.
    :zoom_start:  -> Initial zoom level for the map.
    
    return:      -> A folium.Map object.
    '''
    mapViz = folium.Map(location=center,tiles=Tiles, zoom_start=zoom_start)
    for k,v in dicc.items():
      if ee.image.Image in [type(x) for x in v.values()]:
        folium.TileLayer(
            tiles = v["tile_fetcher"].url_format,
            attr  = 'Google Earth Engine',
            overlay =True,
            name  = k
          ).add_to(mapViz)
      else:
        folium.GeoJson(
        data = v,
        name = k
          ).add_to(mapViz)
    mapViz.add_child(folium.LayerControl())
    if markers:  
      for i in range(0,len(points)):
        folium.Marker([geoData.iloc[i]['lon'], geoData.iloc[i]['lat']]).add_to(mapViz)
    return mapViz


In [9]:
# Select the RGB values from the image to display

In [11]:
mapid = image_colletion_2018[-1].getMapId({
    'bands': ['B4', 'B3', 'B2'], 
    'min': 0, 
    'max': 3000})

In [12]:
farm = Mapdisplay([-33.19458, 19.33449], {'Median Composite':mapid},zoom_start=15)

In [13]:
# on a notebook the farm has to be called indivudually again to display
farm

In [14]:
# Create a polygon spanning a certain orchard on the farm

In [15]:
poly = ee.Geometry.Polygon([[19.3313128875642,-33.19618471068494],
[19.331200234785573,-33.194891883784784],
[19.329692833319204,-33.19471681200812],
[19.329478256598012,-33.195996174617406],
[19.3313128875642,-33.19618471068494]])

In [16]:
# A method used to display the polygon on the selected sentinel image

In [17]:
# this method calculates the average coordiante of a polygon and adds a marker on this coordinate
def AddMapMarkers(geometry, map, marker_name=None):
    
    x = 0.0
    y = 0.0
    length = 0
    for i in geometry['coordinates']:
        length = len(i[:-1])
        for j in i[:-1]:
            x += j[1]
            y += j[0]

    x /= length
    y /= length
    
    folium.Marker([x, y], marker_name).add_to(map)

In [18]:
AddMapMarkers(poly, farm, "a piece of land")
farm

In [19]:
def reduce_image_dictionary(polygon, image):
    ''' Reduce a polygon within an image to a single value in each band of an sentinel 2 image'''
    meanDictionary = image.reduceRegion(**{
        'reducer': ee.Reducer.mean(),
        'geometry': polygon,
        'scale': 30,
        'maxPixels': 1e9
    })
    return meanDictionary

In [20]:
zonalStatistics = reduce_image_dictionary(poly, image_colletion_2018[-1])

In [21]:
print(zonalStatistics.getInfo())

{'B1': 1163.401044965168, 'B10': 12.52137428752375, 'B11': 2225.5178910702975, 'B12': 1242.0945218492716, 'B2': 928.7215009499683, 'B3': 1000.066656111463, 'B4': 881.964059531349, 'B5': 1337.852913236225, 'B6': 2740.0101329955664, 'B7': 3274.9708676377454, 'B8': 3150.3977200759978, 'B8A': 3586.6654528182394, 'B9': 955.5037998733375, 'QA10': 0, 'QA20': 0, 'QA60': 0}


In [22]:
for i in zonalStatistics.getInfo():
    print("Band: {}, value: {}".format(i, zonalStatistics.getInfo()[i]))

Band: B1, value: 1163.401044965168
Band: B10, value: 12.52137428752375
Band: B11, value: 2225.5178910702975
Band: B12, value: 1242.0945218492716
Band: B2, value: 928.7215009499683
Band: B3, value: 1000.066656111463
Band: B4, value: 881.964059531349
Band: B5, value: 1337.852913236225
Band: B6, value: 2740.0101329955664
Band: B7, value: 3274.9708676377454
Band: B8, value: 3150.3977200759978
Band: B8A, value: 3586.6654528182394
Band: B9, value: 955.5037998733375
Band: QA10, value: 0
Band: QA20, value: 0
Band: QA60, value: 0
